In [4]:
!pip install dash plotly


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\WELCOME\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [9]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.figure_factory as ff

In [15]:
# load Dataset
df = pd.read_csv('D:/data_analyst/AnalyticsProjects/projects/employee_data.csv')

# Define a list of specific features
selected_columns = ['MonthlyIncome', 'Age', 'TotalWorkingYears', 'YearsAtCompany', 'Attrition']

# Compute correlation matrix for only selected columns
corr_matrix = df[selected_columns].corr().round(2)

# Create the heatmap
fig_corr = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=list(corr_matrix.columns),
    y=list(corr_matrix.index),
    colorscale='Viridis',
    showscale=True
)


# Create app
app = dash.Dash(__name__)
app.title = 'Employee Attrition Dashboard'

app.layout = html.Div([
    html.H1("Employee Attrition Dashboard", style={"textAlign": "center"}),

    html.Div([
        html.Label('Select Feature:'),
        dcc.Dropdown(
            id='feature-dropdown',
            options=[
                {'label': 'Department - R&D', 'value': 'Department_Research & Development'},
                {'label': 'Job Role - Research Scientist', 'value': 'JobRole_Research Scientist'},
                {'label': 'Gender (Male)', 'value': 'Gender_Male'},
                {'label': 'OverTime', 'value': 'OverTime'},
            ],
            value='OverTime'
        )
    ], style={"width": "50%", "margin": "auto"}),

    dcc.Graph(id='bar-chart'),

    html.H2("Attrition Distribution"),
    dcc.Graph(figure=px.histogram(df, x='Attrition', title='Attrition Count')),

    html.H2("Monthly Income Distribution"),
    dcc.Graph(figure=px.histogram(df, x='MonthlyIncome', nbins=50, title='Income Distribution')),

    html.H2("Job Satisfaction Distribution"),
    dcc.Graph(figure=px.histogram(df, x='JobSatisfaction', nbins=4, title='Job Satisfaction Levels')),

    html.H2("Attrition by Age"),
    dcc.Graph(figure=px.histogram(df, x='Age', color='Attrition', barmode='overlay', title='Attrition Across Age Groups')),

    html.H2("Monthly Income vs. Attrition"),
    dcc.Graph(figure=px.box(df, x='Attrition', y='MonthlyIncome', title='Income Distribution by Attrition')),

    html.H2("Correlation Heatmap for Selected Features"),
    dcc.Graph(figure=fig_corr)
])

# Callback to update dynamic bar chart

@app.callback(
    Output('bar-chart', 'figure'),
    [Input('feature-dropdown', 'value')]
)
def update_chart(selected_feature):
    fig = px.histogram(df, x=selected_feature, color=df['Attrition'].map({0: 'No', 1: 'Yes'}), barmode='group',
                       title=f'Attrition by {selected_feature}')
    return fig

if __name__ == '__main__':
    app.run(debug=True)
